In [ ]:
from pyspark.sql.functions import *
from IPython.core.display import HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from scipy import interpolate
from scipy.stats import gaussian_kde
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import tqdm

%matplotlib inline
BuGrRd_colors = [(0, 0, 1), (0, 1, 0), (1, 0, 0)]
BuGrRd = LinearSegmentedColormap.from_list("BuGrRd", BuGrRd_colors, N=100)
plt.rcParams['animation.embed_limit'] = 2**128
plt.rcParams["animation.html"] = "jshtml"
display(HTML("<style>pre { white-space: pre !important; }</style>"))
sc.setLogLevel("ERROR")
spark.conf.set("spark.sql.crossJoin.enabled", "true")

# Description of the dataset

One file per month is provided as a csv file with the following
features:

- **callsign**: the identifier of the flight displayed on ATC screens
  (usually the first three letters are reserved for an airline: AFR
  for Air France, DLH for Lufthansa, etc.)
- **number**: the commercial number of the flight, when available (the
  matching with the callsign comes from public open API)
- **icao24**: the transponder unique identification number;
- **registration**: the aircraft tail number (when available);
- **typecode**: the aircraft model type (when available);
- **origin**: a four letter code for the origin airport of the flight
  (when available);
- **destination**: a four letter code for the destination airport of
  the flight (when available);
- **firstseen**: the UTC timestamp of the first message received by
  the OpenSky Network;
- **lastseen**: the UTC timestamp of the last message received by the
  OpenSky Network;
- **day**: the UTC day of the last message received by the OpenSky
  Network.

# Simple Data Separation and Exploration

In [ ]:
df = spark.read.csv("/user/s1919377/flights/*", header='true')
df = df.withColumn("firstseen",to_timestamp("firstseen", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("lastseen",to_timestamp("lastseen", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("day",to_timestamp("day", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("longitude_1",col("longitude_1").cast("float")) \
       .withColumn("longitude_2",col("longitude_2").cast("float")) \
       .withColumn("latitude_1",col("latitude_1").cast("float")) \
       .withColumn("latitude_2",col("latitude_2").cast("float")) \
       .withColumn("altitude_1",col("altitude_1").cast("float")) \
       .withColumn("altitude_2",col("altitude_2").cast("float"))
df.show(truncate=False)
df.printSchema()

## Two Days With and Without COVID-19

In [ ]:
day_precovid = df.where(col('day') == lit("2019-08-01")).persist()
day_precovid.show()

In [ ]:
day_covid = df.where(col('day') == lit('2020-04-01')).persist()
day_covid.show()

In [ ]:
two_days_covid_pandas['firstseen'] = pd.to_datetime(two_days_covid_pandas['firstseen'], format="%Y-%m-%d %H:%M:%S")
two_days_covid_pandas['lastseen'] = pd.to_datetime(two_days_covid_pandas['lastseen'], format="%Y-%m-%d %H:%M:%S")
two_days_covid_pandas.head()

# Data Visualization

In [ ]:
cache = {}

In [ ]:
def make_animation(dataframe, steps, interval, bounds=[[-180, -90], [180, 90]], map_resolution='l', airports=None, cache_id=None, density=False):
    times = np.array(dataframe.select(unix_timestamp("firstseen"), unix_timestamp("lastseen")).collect())
    values_lon = np.array(dataframe.select("longitude_1", "longitude_2").collect())
    values_lat = np.array(dataframe.select("latitude_1", "latitude_2").collect())
    
    min_time = np.min(times[:, 0])
    max_time = np.max(times[:, 1])

    time_per_step = (max_time - min_time) / steps
    starts_ends = np.round((times - min_time) / time_per_step).astype(np.int)
    stepped_steps = np.arange(starts_ends.shape[0]) * steps
    
    lon_interp = None
    lat_interp = None
    if cache_id is None or cache_id not in cache or len(cache[cache_id]) < steps:
        adjusted_starts_ends = starts_ends + np.repeat(np.reshape(stepped_steps, (-1, 1)), starts_ends.shape[1], axis=1)

        known_values_lon = np.repeat(values_lon.flatten(), np.array([starts_ends[:, 0], steps - starts_ends[:, 1]]).T.flatten())
        known_values_lat = np.repeat(values_lat.flatten(), np.array([starts_ends[:, 0], steps - starts_ends[:, 1]]).T.flatten())

        unknown_ranges = np.concatenate([np.arange(x, y) for x, y in adjusted_starts_ends])
        known_ranges = np.delete(np.arange(steps * values_lon.shape[0]), unknown_ranges)

        lon_interp = interpolate.interp1d(known_ranges, known_values_lon, fill_value="extrapolate")
        lat_interp = interpolate.interp1d(known_ranges, known_values_lat, fill_value="extrapolate")
    
    # INSTANTIATE THE PLOT
    fig, ax = plt.subplots(figsize=(20, 10))

    # MAP CODE
    m = Basemap(ax=ax, llcrnrlon=bounds[0][0], llcrnrlat=bounds[0][1], urcrnrlon=bounds[1][0], urcrnrlat=bounds[1][1],
               resolution=map_resolution)
    m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
    m.drawmapboundary(fill_color="#DDEEFF")
    m.drawcoastlines()

    # THE SCATTER PLOT ITSELF
    x, y = m(values_lon[:, 0], values_lat[:, 0])
    scatter = None
    norm = None
    bins = 50
    if density:
        d , x_e, y_e = np.histogram2d(x, y, bins = bins, density = True )
        z = interpolate.interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , d , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

        #To be sure to plot all data
        z[np.where(np.isnan(z))] = 0.0

        # Sort the points by density, so that the densest points are plotted last
        idx = z.argsort()
        sorted_x, sorted_y, z = x[idx], y[idx], z[idx]
        norm = matplotlib.colors.Normalize(vmin=0, vmax=z[-1] + np.std(z) * 2)
        scatter = ax.scatter(sorted_x, sorted_y, c=z, s=0.5, zorder=3, cmap=BuGrRd)
    else:
        scatter = ax.scatter(x, y, color='red', s=0.5, zorder=3)
    
    if airports is not None:
        airports_x, airports_y = m(airports['longitude'], airports['latitude'])
        ax.scatter(airports_x, airports_y, color='blue', zorder=4)

    # THE METHOD TO UPDATE THE VALUES IN THE PLOT FOR EACH ANIMATION STEP
    def animate(i):
        data = None
        if cache_id is None:
            vals = i + stepped_steps
            new_x, new_y = m(lon_interp(vals), lat_interp(vals))
            data = np.column_stack((new_x, new_y))
        elif cache_id in cache and len(cache[cache_id]) >= steps:
            data = cache[cache_id][i]
        else:
            vals = i + stepped_steps
            new_x, new_y = m(lon_interp(vals), lat_interp(vals))
            data = np.column_stack((new_x, new_y))
            if cache_id in cache:
                cache[cache_id].append(data)
            else:
                cache[cache_id] = [data]
        
        # UPDATE THE PLOT VALUES
        
        if density:
            new_x = data[:, 0]
            new_y = data[:, 1]

            d , x_e, y_e = np.histogram2d(new_x, new_y, bins = bins, density = True )
            z = interpolate.interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , d , np.vstack([new_x,new_y]).T , method = "splinef2d", bounds_error = False)

            #To be sure to plot all data
            z[np.where(np.isnan(z))] = 0.0

            # Sort the points by density, so that the densest points are plotted last
            idx = z.argsort()
            sorted_x, sorted_y, z = new_x[idx], new_y[idx], z[idx]
            scatter.set_color(BuGrRd(norm(z)))
            data = np.column_stack((sorted_x, sorted_y))
        scatter.set_offsets(data)
        pbar.update(1)
        return scatter
    
    pbar = tqdm(total=steps + 1)

    # CREATE AND RETURN THE ANIMATION
    return FuncAnimation(fig, animate, interval=interval, frames=steps)

## Flights Before COVID-19

In [ ]:
anim_non_covid = make_animation(two_days_non_covid_pandas, 200, 50)
anim_non_covid

## Flights During COVID-19

In [ ]:
anim_covid = make_animation(two_days_covid_pandas, 200, 50)
anim_covid

### In Europe

In [ ]:
europe_bounds = [
    [-24.0, 34.41],
    [49.98, 71.28]
]
europe_precovid = day_precovid.where(((col('longitude_1') >= europe_bounds[0][0]) & (col('longitude_1') <= europe_bounds[1][0])) &
                                                    ((col('latitude_1') >= europe_bounds[0][1]) & (col('latitude_1') <= europe_bounds[1][1])) |
                                                   ((col('longitude_2') >= europe_bounds[0][0]) & (col('longitude_2') <= europe_bounds[1][0])) &
                                                    ((col('latitude_2') >= europe_bounds[0][1]) & (col('latitude_2') <= europe_bounds[1][1]))
                                                   )
europe_precovid.show()

In [ ]:
europe_covid = day_covid.where(((col('longitude_1') >= europe_bounds[0][0]) & (col('longitude_1') <= europe_bounds[1][0])) &
                                                    ((col('latitude_1') >= europe_bounds[0][1]) & (col('latitude_1') <= europe_bounds[1][1])) |
                                                   ((col('longitude_2') >= europe_bounds[0][0]) & (col('longitude_2') <= europe_bounds[1][0])) &
                                                    ((col('latitude_2') >= europe_bounds[0][1]) & (col('latitude_2') <= europe_bounds[1][1]))
                                                   )
europe_covid.show()

In [ ]:
europe_precovid_anim = make_animation(europe_precovid, 100, 50, europe_bounds, 'i', density=True)
europe_precovid_anim

## Plot With Airports

In [ ]:
airport_metadata = spark.read.json('file:///home/s1919377/project/airport_codes.json')
airport_metadata = airport_metadata.withColumn("longitude",split("coordinates", ', ').getItem(0).cast("float")) \
                                   .withColumn("latitude",split("coordinates", ', ').getItem(1).cast("float"))
airport_metadata.show()
airport_metadata.printSchema()

### European Airports

In [ ]:
large_european_airports = airport_metadata[(airport_metadata.continent == 'EU') & (airport_metadata.type == 'large_airport')]

fig, ax = plt.subplots(figsize=(20,10))

m = Basemap(ax=ax, llcrnrlon=europe_bounds[0][0], llcrnrlat=europe_bounds[0][1],
            urcrnrlon=europe_bounds[1][0], urcrnrlat=europe_bounds[1][1],
            resolution='i')
m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
m.drawmapboundary(fill_color="#DDEEFF")
m.drawcoastlines()

x, y = m(large_european_airports['longitude'], large_european_airports['latitude'])
ax.scatter(x, y, color='blue', zorder=3)

for i, row in large_european_airports.iterrows():
    code = row['ident']
    ax.annotate(code, (x[i], y[i]))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

values_lon = europe_flights_non_covid_pd[['longitude_1']].astype(np.float).to_numpy()
values_lat = europe_flights_non_covid_pd[['latitude_1']].astype(np.float).to_numpy()

m = Basemap(ax=ax, llcrnrlon=europe_bounds[0][0], llcrnrlat=europe_bounds[0][1],
            urcrnrlon=europe_bounds[1][0], urcrnrlat=europe_bounds[1][1],
            resolution='i')
m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
m.drawmapboundary(fill_color="#DDEEFF")
m.drawcoastlines()

x, y = m(values_lon.flatten(), values_lat.flatten())
x = x.flatten()
y = y.flatten()

bins = 50

data , x_e, y_e = np.histogram2d(x, y, bins = bins, density = True )
z = interpolate.interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

#To be sure to plot all data
z[np.where(np.isnan(z))] = 0.0

# Sort the points by density, so that the densest points are plotted last
if True :
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

scat = ax.scatter(x, y, c=z, s=0.5, zorder=3, cmap=BuGrRd)

plt.show()

In [ ]:
europe_covid = day_covid.where(((col('longitude_1') >= europe_bounds[0][0]) & (col('longitude_1') <= europe_bounds[1][0])) &
                                                    ((col('latitude_1') >= europe_bounds[0][1]) & (col('latitude_1') <= europe_bounds[1][1])) |
                                                   ((col('longitude_2') >= europe_bounds[0][0]) & (col('longitude_2') <= europe_bounds[1][0])) &
                                                    ((col('latitude_2') >= europe_bounds[0][1]) & (col('latitude_2') <= europe_bounds[1][1]))
                                                   )

In [ ]:
europe_covid_anim = make_animation(europe_covid, 100, 50,
                                   bounds=europe_bounds,
                                   map_resolution='i',
                                   density=True)
europe_covid_anim

## Large European Airports By Origin Count

In [ ]:
large_european_airports = airport_metadata.where((col('continent') == 'EU') & (col('type') == 'large_airport'))
df_europe = df.join(large_european_airports,
                    [(df.origin == large_european_airports.ident) | (df.destination == large_european_airports.ident)],
                    'leftsemi').persist()

In [ ]:
covid_counts = day_covid.groupBy('origin').count().show()

In [ ]:
precovid_counts = day_precovid.groupBy('origin').count()
covid_counts = day_covid.groupBy('origin').count()
large_european_airports = large_european_airports \
    .join(precovid_counts, [large_european_airports.ident == precovid_counts.origin], "inner") \
    .drop("origin") \
    .withColumnRenamed("count","precovid_count") \
    .join(covid_counts, [large_european_airports.ident == covid_counts.origin], "inner") \
    .drop("origin") \
    .withColumnRenamed("count","covid_count")

### Before COVID

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

m = Basemap(ax=ax, llcrnrlon=europe_bounds[0][0], llcrnrlat=europe_bounds[0][1],
            urcrnrlon=europe_bounds[1][0], urcrnrlat=europe_bounds[1][1],
            resolution='i')
m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
m.drawmapboundary(fill_color="#DDEEFF")
m.drawcoastlines()

m.scatter(large_european_airports.select("longitude").collect(),
          large_european_airports.select("latitude").collect(), latlon=True,
          c=np.log10(large_european_airports.select("precovid_count").collect()),
          s=large_european_airports.select("precovid_count").collect(),
          cmap='Reds', alpha=0.5, zorder=3)

plt.show()

### During COVID

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

m = Basemap(ax=ax, llcrnrlon=europe_bounds[0][0], llcrnrlat=europe_bounds[0][1],
            urcrnrlon=europe_bounds[1][0], urcrnrlat=europe_bounds[1][1],
            resolution='i')
m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
m.drawmapboundary(fill_color="#DDEEFF")
m.drawcoastlines()

m.scatter(large_european_airports.select("longitude").collect(),
          large_european_airports.select("latitude").collect(), latlon=True,
          c=np.log10(large_european_airports.select("covid_count").collect()),
          s=large_european_airports.select("covid_count").collect(),
          cmap='Blues', alpha=0.5, zorder=3)

plt.show()

## Animate Airport Density

In [ ]:
df_europe.show()
df_europe.printSchema()

In [ ]:
unique_days = df_europe \
    .select("day") \
    .distinct() \
    .sort(col('day').asc()) \
    .collect()
unique_days = np.array(unique_days).flatten()

In [ ]:
first_day = df_europe.where(col("day") == unique_days[0])
first_day_counts = first_day.groupBy('origin').count()
first_day_densities = large_european_airports \
    .join(first_day_counts, [large_european_airports.ident == first_day.origin], "inner") \
    .drop("origin")
first_day_densities.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

m = Basemap(ax=ax, llcrnrlon=europe_bounds[0][0], llcrnrlat=europe_bounds[0][1],
            urcrnrlon=europe_bounds[1][0], urcrnrlat=europe_bounds[1][1],
            resolution='i')
m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
m.drawmapboundary(fill_color="#DDEEFF")
m.drawcoastlines()

reds = matplotlib.cm.Reds

x, y = m(first_day_densities.select("longitude").collect(),
          first_day_densities.select("latitude").collect())
scat = m.scatter(x, y,
          c=np.log10(first_day_densities.select("count").collect()),
          s=first_day_densities.select("count").collect(),
          cmap="Reds", alpha=0.5, zorder=3)

def density_animate(i):
    n_day = df_europe.where(col("day") == unique_days[i])
    n_day_counts = n_day.groupBy('origin').count()
    n_day_densities = large_european_airports \
        .join(n_day_counts, [large_european_airports.ident == n_day.origin], "inner") \
        .drop("origin")
    x_new = np.array(n_day_densities.select("longitude").collect()).flatten()
    y_new = np.array(n_day_densities.select("latitude").collect()).flatten()
    x_new, y_new = m(x_new, y_new)
    count = np.array(n_day_densities.select("count").collect()).flatten()
    
    scat.set_offsets(np.column_stack((x_new, y_new)))
    scat.set_array(np.log10(count))
    scat.set_sizes(count)
    
    progress.update(1)
    
    return scat

progress = tqdm(total=731)

densities_anim = FuncAnimation(fig, density_animate, interval=50, frames=730)
densities_anim

In [ ]:
writervideo = animation.FFMpegWriter(fps=60)
densities_anim.save('anim.mp4', writer=writervideo)